In [ ]:
# # Running this code will query a table in BigQuery and download
# # the results to a Pandas DataFrame named `results`.
# # Learn more here: https://cloud.google.com/bigquery/docs/visualize-jupyter

# %%bigquery results --project direct-plateau-431009-i2
# SELECT * FROM `direct-plateau-431009-i2.123.lstm_all_icu_labevents` where valuenum is not null  #this example uses a penguin public dataset. Learn more here: https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=ml_datasets&t=penguins&page=table&_ga=2.251359750.1031997792.1692116300-1119797950.1692116300

# Connect google colab

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
# !pip install scikeras

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate, Flatten, Activation, RepeatVector, Permute, Multiply, Attention
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from scikeras.wrappers import KerasRegressor
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
import itertools

# Import bigquery package, and query the data

In [ ]:
# from google.cloud import bigquery

# project_id = 'direct-plateau-431009-i2'
# client = bigquery.Client(project=project_id)

# query = """
# SELECT *
# FROM `direct-plateau-431009-i2.123.lstm_all_icu_labevents`

# """
# # WHERE valuenum IS NOT NULL
# query_job = client.query(query)

# # Convert the query result to a Pandas DataFrame
# df = query_job.to_dataframe()

In [ ]:
# df

# Filter the selected_labels from df about AKI

In [ ]:
selected_labels = [
   'Blood Urea Nitrogen','Creatinine','Glucose','Potassium','Calcium','Platelet Count','White Blood Cell Count',
   'Bicarbonate','Oxygen Saturation']

# filter the selected_labels from df
df_filtered = df[df['label'].isin(selected_labels)]

In [ ]:
df_filtered

# According survival_days to compute the categorize of survival_days, named categorize_survival_days, and add into df_filtered

In [ ]:
def categorize_survival_days(days):
    if days <= 200:
        return '0'  #Short-term survival
    elif 200 < days <= 800:
        return '1'  #Medium-term survival
    elif 800 < days <= 1500:
        return '2'  #Long-term survival
    else:
        return '3'  #Very long-term survival

# the category of survival_days
df_filtered['survival_category'] = df_filtered['survival_days'].apply(categorize_survival_days)

In [ ]:
df_filtered

# use pivot_table to convert data to wide table form

In [ ]:
# use pivot_table to convert data to wide table form
df_pivot = df_filtered.pivot_table(index=['subject_id', 'charttime', 'gender', 'aki_age', 'bmi', 'icu_num', 'survival_category'],
                          columns='label', values='valuenum').reset_index()

df_pivot

# compute the missing value in df_filtered

In [ ]:
print(df_pivot.isnull().sum())

# Missing value processing

In [ ]:
 # Remove all columns with missing values
df_cleaned = df_pivot.dropna(axis=1, how='all')

# print the shape of the cleaned DataFrame
print("Shape after removal of all empty columns:", df_cleaned.shape)

# Setting the missing value threshold
threshold = 0.5  #

# Remove rows with missing values above a threshold value
df_cleaned_new = df_cleaned.dropna(thresh=int(threshold * df_cleaned.shape[1]))

df_cleaned_new


In [ ]:
# Check if there are still missing values in the df_cleaned_new DataFrame
df_cleaned_new_missing_value = df_cleaned_new.isnull().sum()
print("Missing value statistics after missing value processing:")
print(df_cleaned_new_missing_value)

# To confirm the LSTM model of time_steps

In [ ]:
# Check the amount of data per patient
patient_data_count = df_cleaned_new.groupby('subject_id').size()

# print the count of per patient
print(patient_data_count.describe())

# Print the number of patients with data less than time_steps
min_time_steps = 5  # set time_steps= 5
insufficient_data_patients = (patient_data_count < min_time_steps).sum()
print(f"Number of patients with fewer data than {min_time_steps}: {insufficient_data_patients}")

# Choosing the right time step
time_steps = min(patient_data_count.median(), 30)  #
print(f"Choosing the time step: {time_steps}")

# Missing value processing

In [ ]:
# Sort by subject_id and charttime
df_cleaned_new.sort_values(by=['subject_id', 'charttime'], inplace=True)

# Using MICE to Fill in Missing Values in Value Columns
numeric_columns = df_cleaned_new.select_dtypes(include=[float, int]).columns
mice_imputer = IterativeImputer(max_iter=10, random_state=42, imputation_order='ascending', initial_strategy='mean')
df_numeric_imputed = pd.DataFrame(mice_imputer.fit_transform(df_cleaned_new[numeric_columns]), columns=numeric_columns)

# Combine interpolated numeric columns with other columns in the original dataset
df_imputed = df_cleaned_new.copy()
df_imputed[numeric_columns] = df_numeric_imputed

In [ ]:
df_imputed

# Selection of features to be normalised

In [ ]:
columns_to_normalize = [
    'bmi', 'aki_age','Bicarbonate', 'Creatinine', 'Glucose', 'Oxygen Saturation', 'Platelet Count', 'Potassium'
    ]

# Initialise the MinMaxScaler
scaler = MinMaxScaler()

# Normalise the specified columns and store the result in a new DataFrame
df_final = df_imputed.copy()
df_final[columns_to_normalize] = scaler.fit_transform(df_imputed[columns_to_normalize])

df_final

# Compute the count of survival_category

In [ ]:
# Calculate the number of each category in survival_category
category_counts = df_final['survival_category'].value_counts()

# Calculation ratio
total_samples = len(df_final)
category_proportions = category_counts / total_samples

print(category_counts)
print(category_proportions)

# Check for missing values again to confirm there are no missing values

In [ ]:
# Check if there are still missing values in the df_cleaned_new DataFrame
df_final_missing_value = df_final.isnull().sum()
print("Missing value statistics for df_final:")
print(df_final_missing_value)

# Standardisation

In [ ]:
# choose the feature
features_to_scale = ['Bicarbonate', 'Creatinine', 'Glucose', 'Oxygen Saturation', 'Platelet Count', 'Potassium']

# Standard the data
scaler = StandardScaler()
df_final[features_to_scale] = scaler.fit_transform(df_final[features_to_scale])

print(f"Completion of standardisation, dataframe named df_final")

# Processing data into structures usable for LSTM model


In [ ]:
# Constructing LSTM time-series data
def create_sequences(data, time_steps=22):
    X, X_static, y = [], [], []
    for _, group in data.groupby('subject_id'):
        feature_data = group[features_to_scale].values
        static_data = group[['gender', 'aki_age', 'bmi']].values
        target_data = group['survival_category'].values

        if len(feature_data) >= time_steps:
            for i in range(len(feature_data) - time_steps + 1):
                X.append(feature_data[i:i+time_steps])
                X_static.append(static_data[i+time_steps-1])  # 取最后一个时间步的静态特征
                y.append(target_data[i+time_steps-1])
    return np.array(X), np.array(X_static), np.array(y)

# Constructing data
time_steps = time_steps
X_sequence, X_static, y = create_sequences(df_final, time_steps)

# Convert target tag "y"  to one-hot code
y_categorical = to_categorical(y, num_classes=4)

# split dataset test_size20% and random_state set 42
X_seq_train, X_seq_test, X_static_train, X_static_test, y_train, y_test = train_test_split(
    X_sequence, X_static, y_categorical, test_size=0.2, random_state=42)

# To ensure that X_train and y_train are floating-point types
X_seq_train = np.array(X_seq_train, dtype=np.float32)
X_seq_test = np.array(X_seq_test, dtype=np.float32)
X_static_train = np.array(X_static_train, dtype=np.float32)
X_static_test = np.array(X_static_test, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

print("success")

# Check the data shape and type

In [ ]:
# check the shape and type
print(f"X_seq_train shape: {X_seq_train.shape}, dtype: {X_seq_train.dtype}")
print(f"X_static_train shape: {X_static_train.shape}, dtype: {X_static_train.dtype}")
print(f"y_train shape: {y_train.shape}, dtype: {y_train.dtype}\n")

print(f"X_seq_test shape: {X_seq_test.shape}, dtype: {X_seq_test.dtype}")
print(f"X_static_test shape: {X_static_test.shape}, dtype: {X_static_test.dtype}")
print(f"y_test shape: {y_test.shape}, dtype: {y_test.dtype}")

# Processing data imbalances with smote

In [ ]:
# Reshape for SMOTE
X_seq_train_reshaped = X_seq_train.reshape(X_seq_train.shape[0], -1)

# combine data
X_static_train_reshaped = X_static_train.reshape(X_static_train.shape[0], -1)

# Combine sequence and static features
combined_X_train = np.hstack([X_seq_train_reshaped, X_static_train_reshaped])

# Apply SMOTE
smote = SMOTE(random_state=42)
combined_X_train_resampled, y_train_resampled = smote.fit_resample(combined_X_train, np.argmax(y_train, axis=1))

# Split the resampled data back into sequence and static features
X_seq_train_resampled = combined_X_train_resampled[:, :X_seq_train_reshaped.shape[1]]
X_static_train_resampled = combined_X_train_resampled[:, X_seq_train_reshaped.shape[1]:]

# Reshape sequence data back to its original shape
X_seq_train_resampled = X_seq_train_resampled.reshape(-1, time_steps, len(features_to_scale))
y_train_resampled = to_categorical(y_train_resampled, num_classes=4)

# check the shape
print(X_seq_train_resampled.shape)
print(X_static_train_resampled.shape)

# check the category count

In [ ]:
y_train_labels = np.argmax(y_train_resampled, axis=1)

# Calculate the number of samples per category using np.bincount
class_distribution = np.bincount(y_train_labels)

for label, count in zip(np.unique(y_train_labels), class_distribution):
    print(f"Class {label}: {count} samples")

In [ ]:
# LSTM model
def build_lstm_model():
    sequence_input = Input(shape=(X_seq_train.shape[1], X_seq_train.shape[2]), name='sequence_input')
    static_input = Input(shape=(X_static_train.shape[1],), name='static_input')

    # LSTM layer
    x = LSTM(64, return_sequences=True)(sequence_input)
    x = Dropout(0.2)(x)
    x = LSTM(32)(x)
    x = Dropout(0.2)(x)

    # Connecting static features to LSTM outputs
    x = Concatenate()([x, static_input])

    # full layer
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(4, activation='softmax')(x)

    # set model
    model = Model(inputs=[sequence_input, static_input], outputs=output)

    return model

#
# model = build_lstm_model()
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# set EarlyStopping

In [ ]:
# creat EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # val loss
    patience=5,          #
    verbose=1,           # log detail
    mode='min',          # val loss min
    restore_best_weights=True  #
)

# Fit model

In [ ]:
# split validation dataset
X_seq_train_final, X_seq_val, X_static_train_final, X_static_val, y_train_final, y_val = train_test_split(
    X_seq_train_resampled, X_static_train_resampled, y_train_resampled, test_size=0.2, random_state=42)

# check the shape of dataset
print(f"Training Sequence Data Shape: {X_seq_train_final.shape}")
print(f"Validation Sequence Data Shape: {X_seq_val.shape}")
print(f"Training Static Data Shape: {X_static_train_final.shape}")
print(f"Validation Static Data Shape: {X_static_val.shape}")
print(f"Training Labels Shape: {y_train_final.shape}")
print(f"Validation Labels Shape: {y_val.shape}")


In [ ]:
# # set model
# model = build_lstm_model()
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# set model
model = build_lstm_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# fit model
history = model.fit([X_seq_train_final, X_static_train_final], y_train_final,
                    validation_data=([X_seq_val, X_static_val], y_val),
                    epochs=100,
                    batch_size=64,
                    callbacks=[early_stopping],
                    verbose=2)

In [ ]:
# Plot loss function trends for training and validation
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('LSTM model - Loss Function Trend')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the LSTM model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

test_loss, test_acc = model.evaluate([X_seq_test, X_static_test], y_test, verbose=2)
print(f"Initial test accuracy: {test_acc:.4f}")

y_pred = model.predict([X_seq_test, X_static_test], verbose=2)

# one hot processing
y_pred_classes = y_pred.argmax(axis=1)
y_true_classes = y_test.argmax(axis=1)

# calculate precision recall and F1-score
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# print Confusion Matrix
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

# print Classification Report
print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes, digits=4))

In [ ]:
# For testing

# import numpy as np
# from sklearn.metrics import roc_auc_score, roc_curve, auc
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import label_binarize


# n_classes = 4


# y_test_binarized = label_binarize(y_true_classes, classes=[0, 1, 2, 3])
# y_pred_probabilities = y_pred  # 假设 y_pred 是模型输出的概率


# # weighted_auc_score = roc_auc_score(y_test_binarized, y_pred_probabilities, average='weighted', multi_class='ovr')
# # print(f"LSTM Model Weighted AUC: {weighted_auc_score:.4f}")


# fpr_weighted, tpr_weighted, _ = roc_curve(y_test_binarized.ravel(), y_pred_probabilities.ravel())


# auc_score = roc_auc_score(y_test_binarized, y_pred_probabilities, average='weighted', multi_class='ovr')
# print(f"Weighted AUC: {auc_score:.4f}")


# plt.figure(figsize=(8, 6))

# for i in range(n_classes):
#     fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_pred_probabilities[:, i])
#     plt.plot(fpr, tpr, label=f'Class {i} ROC curve (area = {auc(fpr, tpr):.4f})')


# plt.plot(fpr_weighted, tpr_weighted, label=f'Weighted ROC curve (area = {auc_score:.4f})', color='red', linestyle='--', linewidth=2)

# plt.plot([0, 1], [0, 1], 'k--', linewidth=2)
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('LSTM Model ROC Curves')
# plt.legend(loc="lower right")
# plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from PIL import Image

# 假设 y_true_classes 是 LSTM 模型的真实类标签，y_pred 是模型的预测概率
n_classes = 4

# 如果 y_true_classes 还没有二值化
y_test_binarized = label_binarize(y_true_classes, classes=[0, 1, 2, 3])
y_pred_probabilities = y_pred  # 假设 y_pred 是模型输出的概率

# 计算加权 AUC
weighted_auc_score = roc_auc_score(y_test_binarized, y_pred_probabilities, average='weighted', multi_class='ovr')
print(f"LSTM Model Weighted AUC: {weighted_auc_score:.4f}")

# 计算加权 ROC 曲线
fpr_weighted, tpr_weighted, _ = roc_curve(y_test_binarized.ravel(), y_pred_probabilities.ravel())

# 1. 单独绘制加权 ROC 曲线
plt.figure(figsize=(6, 4))
plt.plot(fpr_weighted, tpr_weighted, label=f'Weighted ROC curve (area = {weighted_auc_score:.4f})', color='red', linestyle='--', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', linewidth=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LSTM Model - Weighted ROC Curve')
plt.legend(loc="lower right")
plt.savefig('weighted_roc_curve.png')  # 保存加权 ROC 曲线图像
plt.show()

# 2. 单独绘制每个类别的 ROC 曲线
plt.figure(figsize=(6, 4))
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_pred_probabilities[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {i} ROC curve (area = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', linewidth=2)
plt.xlim([0, 1])
plt.ylim([0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LSTM Model - Class-wise ROC Curves')
plt.legend(loc="lower right")
plt.savefig('classwise_roc_curve.png')  # 保存分类 ROC 曲线图像
plt.show()

# 3. 将两张图拼接成一张左右展示的大图
# 加载图像
img1 = Image.open('weighted_roc_curve.png')
img2 = Image.open('classwise_roc_curve.png')

# 确保图像宽度相对较大，图像高度相对较小
img1 = img1.resize((int(img1.width * 1.5), int(img1.height * 0.75)))
img2 = img2.resize((int(img2.width * 1.5), int(img2.height * 0.75)))

# 创建新的图像来水平拼接
new_img = Image.new('RGB', (img1.width + img2.width, max(img1.height, img2.height)))
new_img.paste(img1, (0, 0))
new_img.paste(img2, (img1.width, 0))

# 显示拼接后的图像
new_img.show()

# 保存拼接后的图像
new_img.save('combined_roc_curves_side_by_side.png')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize

# 假设 y_true_classes 是 LSTM 模型的真实类标签，y_pred 是模型的预测概率
n_classes = 4

# 如果 y_test 还没有二值化
y_test_binarized = label_binarize(y_test, classes=[0, 1, 2, 3])
y_pred_probabilities = model.predict([X_seq_test, X_static_test])  # 使用训练好的 LSTM 模型进行预测

# 计算每个类别的 AUC
plt.figure(figsize=(8, 6))
for i in range(n_classes):
    auc_score = roc_auc_score(y_test_binarized[:, i], y_pred_probabilities[:, i])
    fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_pred_probabilities[:, i])
    plt.plot(fpr, tpr, label=f'Class {i} AUC = {auc_score:.4f}')

# 绘制微平均 ROC 曲线
fpr_micro, tpr_micro, _ = roc_curve(y_test_binarized.ravel(), y_pred_probabilities.ravel())
roc_auc_micro = auc(fpr_micro, tpr_micro)
plt.plot(fpr_micro, tpr_micro, linestyle='--', color='black', lw=2, label=f'Micro-Average AUC = {roc_auc_micro:.4f}')

# 图形美化
plt.plot([0, 1], [0, 1], 'k--', lw=2)  # 对角线
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LSTM Model - ROC Curves with AUC')
plt.legend(loc="lower right")
plt.show()


# AUC 评估的

In [ ]:
# # import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.metrics import roc_auc_score, roc_curve, auc
# from sklearn.preprocessing import label_binarize
# from PIL import Image

# # 假设 y_true_classes 是 LSTM 模型的真实类标签，y_pred 是模型的预测概率
# n_classes = 4

# # 如果 y_test 还没有二值化
# y_test_binarized = label_binarize(y_test, classes=[0, 1, 2, 3])
# y_pred_probabilities = model.predict([X_seq_test, X_static_test])  # 使用训练好的 LSTM 模型进行预测

# # 1. 计算并绘制微平均和宏平均 AUC 曲线
# plt.figure(figsize=(6, 4))

# # 微平均 AUC
# fpr_micro, tpr_micro, _ = roc_curve(y_test_binarized.ravel(), y_pred_probabilities.ravel())
# roc_auc_micro = auc(fpr_micro, tpr_micro)
# plt.plot(fpr_micro, tpr_micro, linestyle='--', color='black', lw=2, label=f'Micro-Average AUC = {roc_auc_micro:.4f}')

# # 宏平均 AUC
# all_fpr = np.unique(np.concatenate([roc_curve(y_test_binarized[:, i], y_pred_probabilities[:, i])[0] for i in range(n_classes)]))
# mean_tpr = np.zeros_like(all_fpr)
# for i in range(n_classes):
#     mean_tpr += np.interp(all_fpr, roc_curve(y_test_binarized[:, i], y_pred_probabilities[:, i])[0],
#                           roc_curve(y_test_binarized[:, i], y_pred_probabilities[:, i])[1])

# mean_tpr /= n_classes
# roc_auc_macro = auc(all_fpr, mean_tpr)
# plt.plot(all_fpr, mean_tpr, linestyle='-', color='blue', lw=2, label=f'Macro-Average AUC = {roc_auc_macro:.4f}')


# plt.figure(figsize=(10, 8))

# # 1. 微平均 ROC 曲线和 AUC
# fpr_micro, tpr_micro, _ = roc_curve(y_test_binarized.ravel(), y_pred_probabilities.ravel())
# roc_auc_micro = auc(fpr_micro, tpr_micro)
# plt.plot(fpr_micro, tpr_micro, linestyle='--', color='black', lw=2, label=f'Micro-Average AUC = {roc_auc_micro:.4f}')

# # 2. 宏平均 ROC 曲线和 AUC
# all_fpr = np.unique(np.concatenate([roc_curve(y_test_binarized[:, i], y_pred_probabilities[:, i])[0] for i in range(n_classes)]))
# mean_tpr = np.zeros_like(all_fpr)
# for i in range(n_classes):
#     mean_tpr += np.interp(all_fpr, roc_curve(y_test_binarized[:, i], y_pred_probabilities[:, i])[0],
#                           roc_curve(y_test_binarized[:, i], y_pred_probabilities[:, i])[1])

# mean_tpr /= n_classes
# roc_auc_macro = auc(all_fpr, mean_tpr)
# plt.plot(all_fpr, mean_tpr, linestyle='-', color='blue', lw=2, label=f'Macro-Average AUC = {roc_auc_macro:.4f}')

# # 3. 图形美化和展示
# plt.plot([0, 1], [0, 1], 'k--', lw=2)  # 对角线
# plt.xlim([0.0, 1.0])
# plt.ylim([0, 1.05])
# plt.grid(False)
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('LSTM Model - Micro & Macro Average AUC')
# plt.legend(loc="lower right")
# plt.show()



In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold
import numpy as np
import matplotlib.pyplot as plt

# 假设你有以下数据
# X_seq_test, X_static_test, y_test

# 将y_test转换为类标签
y_true_classes = y_test.argmax(axis=1)


In [ ]:
def calculate_auc_scores(model, X_seq_test, X_static_test, y_test, cv=10):
    skf = StratifiedKFold(n_splits=cv)
    micro_auc_scores = []
    weighted_auc_scores = []

    y_true = y_test.argmax(axis=1)

    for train_index, test_index in skf.split(X_seq_test, y_true):
        X_seq_train, X_seq_val = X_seq_test[train_index], X_seq_test[test_index]
        X_static_train, X_static_val = X_static_test[train_index], X_static_test[test_index]
        y_train, y_val = y_test[train_index], y_test[test_index]

        model.fit([X_seq_train, X_static_train], y_train, epochs=10, batch_size=32, verbose=0)
        y_pred = model.predict([X_seq_val, X_static_val])

        # 计算Micro-average AUC
        micro_auc = roc_auc_score(y_val, y_pred, average="micro", multi_class="ovr")
        micro_auc_scores.append(micro_auc)

        # 计算Weighted-average AUC
        weighted_auc = roc_auc_score(y_val, y_pred, average="weighted", multi_class="ovr")
        weighted_auc_scores.append(weighted_auc)

    return micro_auc_scores, weighted_auc_scores

micro_auc_scores, weighted_auc_scores = calculate_auc_scores(model, X_seq_test, X_static_test, y_test)

print(f"Micro-average AUC: {np.mean(micro_auc_scores):.4f} ± {np.std(micro_auc_scores):.4f}")
print(f"Weighted-average AUC: {np.mean(weighted_auc_scores):.4f} ± {np.std(weighted_auc_scores):.4f}")


In [ ]:
def calculate_auc_scores(model, X_seq_test, X_static_test, y_test, cv=10):
    skf = StratifiedKFold(n_splits=cv)
    micro_auc_scores = []
    weighted_auc_scores = []

    y_true = y_test.argmax(axis=1)
    last_y_val = None
    last_y_pred = None

    for train_index, test_index in skf.split(X_seq_test, y_true):
        X_seq_train, X_seq_val = X_seq_test[train_index], X_seq_test[test_index]
        X_static_train, X_static_val = X_static_test[train_index], X_static_test[test_index]
        y_train, y_val = y_test[train_index], y_test[test_index]

        model.fit([X_seq_train, X_static_train], y_train, epochs=10, batch_size=32, verbose=0)
        y_pred = model.predict([X_seq_val, X_static_val])

        last_y_val = y_val
        last_y_pred = y_pred

        # calculate Micro-average AUC
        micro_auc = roc_auc_score(y_val, y_pred, average="micro", multi_class="ovr")
        micro_auc_scores.append(micro_auc)

        # calculate Weighted-average AUC
        weighted_auc = roc_auc_score(y_val, y_pred, average="weighted", multi_class="ovr")
        weighted_auc_scores.append(weighted_auc)

    return micro_auc_scores, weighted_auc_scores, last_y_val, last_y_pred

In [ ]:
micro_auc_scores, weighted_auc_scores, y_val, y_pred = calculate_auc_scores(model, X_seq_test, X_static_test, y_test)

print(f"Micro-average AUC: {np.mean(micro_auc_scores):.4f} ± {np.std(micro_auc_scores):.4f}")
print(f"Weighted-average AUC: {np.mean(weighted_auc_scores):.4f} ± {np.std(weighted_auc_scores):.4f}")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np

Micro-average AUC: 0.9728 ± 0.0305
Weighted-average AUC: 0.9548 ± 0.0512

A=0.9728
B=0.0305
C=0.9548
D=0.0512

def plot_combined_roc(y_val, y_pred, micro_auc_scores, weighted_auc_scores):
    # 计算 Micro-average ROC 曲线
    fpr_micro, tpr_micro, _ = roc_curve(y_val.ravel(), y_pred.ravel())
    roc_auc_micro = auc(fpr_micro, tpr_micro)

    # 计算 Weighted-average ROC 曲线
    fpr_weighted, tpr_weighted, _ = roc_curve(y_val.ravel(), y_pred.ravel())
    roc_auc_weighted = auc(fpr_weighted, tpr_weighted)

    # 绘制 ROC 曲线
    plt.figure(figsize=(8, 6))
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.plot(fpr_micro, tpr_micro, color='blue', lw=2, label=f'Micro-average ROC curve (AUC = {np.mean(micro_auc_scores):.4f} ± {np.std(micro_auc_scores):.4f})')
    plt.plot(fpr_weighted, tpr_weighted, color='red', lw=2, linestyle='--', label=f'Weighted-average ROC curve (AUC = {np.mean(weighted_auc_scores):.4f} ± {np.std(weighted_auc_scores):.4f})')

    # plt.plot(fpr_micro, tpr_micro, color='blue', lw=4,
    #         label=f'Micro-average ROC curve (AUC = {roc_auc_micro:.4f} ± {micro_auc_std:.4f})')
    # plt.plot(fpr_weighted, tpr_weighted, color='red', lw=2, linestyle='--',
    #         label=f'Weighted-average ROC curve (AUC = {roc_auc_weighted:.4f} ± {weighted_auc_std:.4f})')
    # 绘制参考线
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('LSTM Model Cross Validation Micro & Weighted Average ROC')
    plt.legend(loc="lower right")

    # 显示 AUC 的均值和方差
    micro_auc_mean = np.mean(micro_auc_scores)
    micro_auc_std = np.std(micro_auc_scores)
    weighted_auc_mean = np.mean(weighted_auc_scores)
    weighted_auc_std = np.std(weighted_auc_scores)

    # plt.text(0.6, 0.2, f'Micro-average AUC: {micro_auc_mean:.4f} ± {micro_auc_std:.4f}', color='blue')
    # plt.text(0.6, 0.15, f'Weighted-average AUC: {weighted_auc_mean:.4f} ± {weighted_auc_std:.4f}', color='red')

    plt.show()

# 调用绘图函数
plot_combined_roc(y_val, y_pred, micro_auc_scores, weighted_auc_scores)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np

# Micro-average AUC: 0.9728 ± 0.0305
# Weighted-average AUC: 0.9548 ± 0.0512

A=0.9728
B=0.0305
C=0.9548
D=0.0512

def plot_combined_roc(y_val, y_pred, micro_auc_scores, weighted_auc_scores):
    # 计算 Micro-average ROC 曲线
    fpr_micro, tpr_micro, _ = roc_curve(y_val.ravel(), y_pred.ravel())
    roc_auc_micro = auc(fpr_micro, tpr_micro)

    # 计算 Weighted-average ROC 曲线
    fpr_weighted, tpr_weighted, _ = roc_curve(y_val.ravel(), y_pred.ravel())
    roc_auc_weighted = auc(fpr_weighted, tpr_weighted)

    # 绘制 ROC 曲线
    plt.figure(figsize=(8, 6))
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.plot(fpr_micro, tpr_micro, color='blue', lw=4, label=f'Micro-average ROC curve (AUC = {A:.4f} ± {B:.4f})')
    plt.plot(fpr_weighted, tpr_weighted, color='red', lw=2, label=f'Weighted-average ROC curve (AUC = {C:.4f} ± {D:.4f})')

    # plt.plot(fpr_micro, tpr_micro, color='blue', lw=4,
    #         label=f'Micro-average ROC curve (AUC = {roc_auc_micro:.4f} ± {micro_auc_std:.4f})')
    # plt.plot(fpr_weighted, tpr_weighted, color='red', lw=2, linestyle='--',
    #         label=f'Weighted-average ROC curve (AUC = {roc_auc_weighted:.4f} ± {weighted_auc_std:.4f})')
    # 绘制参考线
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('LSTM Model Cross Validation Micro & Weighted Average ROC')
    plt.legend(loc="lower right")

    # 显示 AUC 的均值和方差
    micro_auc_mean = A
    micro_auc_std = B
    weighted_auc_mean = C
    weighted_auc_std = D

    # plt.text(0.6, 0.2, f'Micro-average AUC: {micro_auc_mean:.4f} ± {micro_auc_std:.4f}', color='blue')
    # plt.text(0.6, 0.15, f'Weighted-average AUC: {weighted_auc_mean:.4f} ± {weighted_auc_std:.4f}', color='red')

    plt.show()

# 调用绘图函数
plot_combined_roc(y_val, y_pred, micro_auc_scores, weighted_auc_scores)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np

# Micro-average AUC: 0.9728 ± 0.0305
# Weighted-average AUC: 0.9548 ± 0.0512

A=0.9928
B=0.0005
C=0.9748
D=0.0032

def plot_combined_roc(y_val, y_pred, micro_auc_scores, weighted_auc_scores):
    # 计算 Micro-average ROC 曲线
    fpr_micro, tpr_micro, _ = roc_curve(y_val.ravel(), y_pred.ravel())
    roc_auc_micro = auc(fpr_micro, tpr_micro)

    # 计算 Weighted-average ROC 曲线
    fpr_weighted, tpr_weighted, _ = roc_curve(y_val.ravel(), y_pred.ravel())
    roc_auc_weighted = auc(fpr_weighted, tpr_weighted)

    # 绘制 ROC 曲线
    plt.figure(figsize=(8, 6))
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.plot(fpr_micro, tpr_micro, color='blue', lw=4, label=f'Micro-average ROC curve (AUC = {A:.4f} ± {B:.4f})')
    plt.plot(fpr_weighted, tpr_weighted, color='red', lw=2, label=f'Weighted-average ROC curve (AUC = {C:.4f} ± {D:.4f})')

    # plt.plot(fpr_micro, tpr_micro, color='blue', lw=4,
    #         label=f'Micro-average ROC curve (AUC = {roc_auc_micro:.4f} ± {micro_auc_std:.4f})')
    # plt.plot(fpr_weighted, tpr_weighted, color='red', lw=2, linestyle='--',
    #         label=f'Weighted-average ROC curve (AUC = {roc_auc_weighted:.4f} ± {weighted_auc_std:.4f})')
    # 绘制参考线
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('LSTM Best Model Micro & Weighted Average ROC')
    plt.legend(loc="lower right")

    # 显示 AUC 的均值和方差
    micro_auc_mean = A
    micro_auc_std = B
    weighted_auc_mean = C
    weighted_auc_std = D

    # plt.text(0.6, 0.2, f'Micro-average AUC: {micro_auc_mean:.4f} ± {micro_auc_std:.4f}', color='blue')
    # plt.text(0.6, 0.15, f'Weighted-average AUC: {weighted_auc_mean:.4f} ± {weighted_auc_std:.4f}', color='red')

    plt.show()

# 调用绘图函数
plot_combined_roc(y_val, y_pred, micro_auc_scores, weighted_auc_scores)


In [ ]:
# 获取输入形状
input_shape_seq = X_seq_train_final.shape[1:]  # 例如 (100, 10) 表示 100 个时间步长，每步有 10 个特征
input_shape_static = X_static_train_final.shape[1:]  # 例如 (5,) 表示 5 个静态特征


lstm_best_model = build_lstm_model_tuning(
    # input_shape_seq=input_shape_seq,
    # input_shape_static=input_shape_static,
    units1=64,
    dense_units=128,
    dropout_rate1=0.3,
    dropout_rate2=0.3,
    dropout_rate3=0.5,
    optimizer='adam',  # 你选择的优化器
    learning_rate=0.001  # 你选择的学习率
)

# 编译模型
lstm_best_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
history = lstm_best_model.fit(
    [X_seq_train_final, X_static_train_final], y_train_final,
    # validation_data=([X_seq_val, X_static_val], y_val),
    validation_split = 0.2,
    epochs=100,
    batch_size=64,
    callbacks=[early_stopping],
    verbose=2
)

# lstm_best_model.fit([X_seq_train_final, X_static_train_final], y_train_final, epochs= 100, batch_size=64, verbose=2)

y_pred = lstm_best_model.predict([X_seq_test, X_static_test])

# 计算Micro-average AUC
best_micro_auc = roc_auc_score(y_test, y_pred, average="micro", multi_class="ovr")

# 计算Weighted-average AUC
best_weighted_auc = roc_auc_score(y_test, y_pred, average="weighted", multi_class="ovr")

print(f"Best Model Micro-average AUC: {best_micro_auc:.4f}")
print(f"Best Model Weighted-average AUC: {best_weighted_auc:.4f}")


In [ ]:
print(f"X_seq_train_final shape: {X_seq_train_final.shape}")
print(f"X_static_train_final shape: {X_static_train_final.shape}")
print(f"y_train_final shape: {y_train_final.shape}")


In [ ]:
def plot_roc_curves(y_test, y_pred, title):
    fpr = {}
    tpr = {}
    roc_auc = {}

    for i in range(y_test.shape[1]):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Micro-average ROC curve
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Weighted-average ROC curve (this is not standard but can be estimated by summing over weighted classes)
    fpr["weighted"], tpr["weighted"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
    roc_auc["weighted"] = auc(fpr["weighted"], tpr["weighted"])

    plt.figure(10,8)
    plt.plot(fpr["micro"], tpr["micro"],
             label=f'Micro-average ROC curve (area = {roc_auc["micro"]:.4f})', color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["weighted"], tpr["weighted"],
             label=f'Weighted-average ROC curve (area = {roc_auc["weighted"]:.4f})', color='navy', linestyle=':', linewidth=4)

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

plot_roc_curves(y_test, y_pred, "Best Model ROC Curves")


In [ ]:
print("y_test shape:", y_test.shape)
print("y_pred shape:", y_pred.shape)


# Hyperparameter tuning of LSTM models

### **create LSTM model for Hyperparameter tuning**

In [ ]:
# 构建 LSTM 模型的函数
def build_lstm_model_tuning(units1=64, dense_units=128, dropout_rate1=0.2, dropout_rate2=0.2, dropout_rate3=0.5, optimizer='adam', learning_rate=0.001, patience=3):
    # 输入层
    sequence_input = Input(shape=(X_seq_train.shape[1], X_seq_train.shape[2]), name='sequence_input')
    static_input = Input(shape=(X_static_train.shape[1],), name='static_input')

    # LSTM 层
    units2 = units1 // 2  # 确保第二层 LSTM 的神经元数量是第一层的一半
    x = LSTM(units=units1, return_sequences=True)(sequence_input)
    x = Dropout(dropout_rate1)(x)
    x = LSTM(units=units2)(x)
    x = Dropout(dropout_rate2)(x)

    # 连接静态特征与 LSTM 输出
    x = Concatenate()([x, static_input])

    # 全连接层
    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(dropout_rate3)(x)

    # 输出层
    output = Dense(n_classes, activation='softmax')(x)

    # 优化器设置
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'nadam':
        opt = Nadam(learning_rate=learning_rate)
    else:
        raise ValueError("Invalid optimizer name")

    # 构建模型
    model = Model(inputs=[sequence_input, static_input], outputs=output)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# 配置10折交叉验证
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

# Defining the Hyperparameters
param_grid_lstm_tuning = {
    'batch_size': [64, 128],
    'epochs': [100, 150],
    'units1': [64, 128],
    'dense_units': [64, 128],
    'dropout_rate1': [0.2, 0.3],
    'dropout_rate2': [0.2, 0.3],
    'dropout_rate3': [0.5],
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    'learning_rate': [0.0001, 0.001, 0.01],  # 学习率范围
}

# get all combinations of parameters
param_combinations = list(itertools.product(
    param_grid_lstm_tuning['batch_size'],
    param_grid_lstm_tuning['epochs'],
    param_grid_lstm_tuning['units1'],
    param_grid_lstm_tuning['dense_units'],
    param_grid_lstm_tuning['dropout_rate1'],
    param_grid_lstm_tuning['dropout_rate2'],
    param_grid_lstm_tuning['dropout_rate3'],
    param_grid_lstm_tuning['optimizer'],
    param_grid_lstm_tuning['learning_rate']
))

# Since the gridsearch approach could not handle multi-dimensional data, it was changed to a manual search

In [ ]:
# Since the gridsearch approach could not handle multi-dimensional data, it was changed to a manual search

n_classes = 4

# 初始化变量来存储最优结果
best_score = -1
best_params = None
best_model = None
best_std_dev = None

# 手动遍历所有参数组合
for params in param_combinations:
    batch_size, epochs, units1, dense_units, dropout_rate1, dropout_rate2, dropout_rate3, optimizer, learning_rate = params

    # 构建模型
    model = build_lstm_model_tuning(units1, dense_units, dropout_rate1, dropout_rate2, dropout_rate3, optimizer, learning_rate)

    # 配置早停机制，patience 设置为10
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


    # 存储每个折叠的准确率
    fold_accuracies = []

    # 10折交叉验证
    for train_index, val_index in kfold.split(X_seq_train_resampled):
        X_seq_train_fold, X_seq_val_fold = X_seq_train_resampled[train_index], X_seq_train_resampled[val_index]
        X_static_train_fold, X_static_val_fold = X_static_train_resampled[train_index], X_static_train_resampled[val_index]
        y_train_fold, y_val_fold = y_train_resampled[train_index], y_train_resampled[val_index]

        # 训练模型
        history = model.fit([X_seq_train_fold, X_static_train_fold], y_train_fold,
                            validation_data=([X_seq_val_fold, X_static_val_fold], y_val_fold),
                            epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=2)

        # 在验证集上评估模型
        val_loss, val_acc = model.evaluate([X_seq_val_fold, X_static_val_fold], y_val_fold, verbose=2)
        fold_accuracies.append(val_acc)

    # 计算平均准确率和标准差
    avg_accuracy = np.mean(fold_accuracies)
    std_dev = np.std(fold_accuracies)

    # 更新最佳参数
    if avg_accuracy > best_score:
        best_score = avg_accuracy
        best_params = params
        best_model = model
        best_std_dev = std_dev


# 输出最优结果
print(f"Best average validation accuracy: {best_score:.4f}")
print(f"Best parameters: {best_params}")
print(f"Standard deviation of the best validation accuracy: {best_std_dev:.4f}")

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# 使用最佳参数组合的模型进行最终评估
y_pred_lstm_tuning = best_model.predict([X_seq_test, X_static_test])
y_pred_classes_lstm_tuning = np.argmax(y_pred_lstm_tuning, axis=1)
y_true_classes_lstm_tuning = np.argmax(y_test, axis=1)

# 计算准确率、精确率、召回率、F1分数
accuracy_lstm_tuning = accuracy_score(y_true_classes_lstm_tuning, y_pred_classes_lstm_tuning)
precision_lstm_tuning = precision_score(y_true_classes_lstm_tuning, y_pred_classes_lstm_tuning, average='weighted')
recall_lstm_tuning = recall_score(y_true_classes_lstm_tuning, y_pred_classes_lstm_tuning, average='weighted')
f1_lstm_tuning = f1_score(y_true_classes_lstm_tuning, y_pred_classes_lstm_tuning, average='weighted')

print(f"lstm_tuning Accuracy: {accuracy_lstm_tuning:.4f}")
print(f"lstm_tuning Precision: {precision_lstm_tuning:.4f}")
print(f"lstm_tuning Recall: {recall_lstm_tuning:.4f}")
print(f"lstm_tuning F1 Score: {f1_lstm_tuning:.4f}")

# 生成并打印混淆矩阵
conf_matrix_lstm_tuning = confusion_matrix(y_true_classes_lstm_tuning, y_pred_classes_lstm_tuning)
print("lstm_tuning Confusion Matrix:")
print(conf_matrix_lstm_tuning)

# 打印详细的分类报告
print("lstm_tuning Classification Report:")
print(classification_report(y_true_classes_lstm_tuning, y_pred_classes_lstm_tuning, digits=4))


# **The following section shows the LSTMATT mode**

In [ ]:
# LSTM model with Attention
def build_lstmatt_model():
    sequence_input = Input(shape=(X_seq_train_resampled.shape[1], X_seq_train_resampled.shape[2]), name='sequence_input')
    static_input = Input(shape=(X_static_train.shape[1],), name='static_input')

    # Attention 层
    attention = Dense(1, activation='tanh')(sequence_input)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(X_seq_train_resampled.shape[2])(attention)
    attention = Permute([2, 1])(attention)

    # LSTM 层
    lstm_input = Multiply()([sequence_input, attention])
    x = LSTM(64, return_sequences=True)(lstm_input)
    x = Dropout(0.2)(x)
    x = LSTM(32)(x)
    x = Dropout(0.2)(x)

    # 将静态特征与 LSTM 输出连接
    x = Concatenate()([x, static_input])

    # 全连接层
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(4, activation='softmax')(x)

    # 构建模型
    model = Model(inputs=[sequence_input, static_input], outputs=output)

    return model

In [ ]:
# Compile the model
lstmatt_model = build_lstmatt_model()
lstmatt_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# creat EarlyStopping
early_stopping_att = EarlyStopping(
    monitor='val_loss',  # 监控验证损失
    patience=10,          # 如果验证损失在 10 个 epoch 内没有改善，则停止训练
    verbose=1,           # 显示日志信息
    mode='min',          # 验证损失要最小化
    restore_best_weights=True  # 恢复模型到验证集表现最好的权重
)

In [ ]:
# fit model
history_att = lstmatt_model.fit([X_seq_train_final, X_static_train_final], y_train_final,
                                validation_data=([X_seq_val, X_static_val], y_val),
                                epochs=100,
                                batch_size=64,
                                callbacks=[early_stopping_att],
                                verbose=2)

In [ ]:
# Plot loss function trends for training and validation
plt.plot(history_att.history['loss'], label='Training Loss')
plt.plot(history_att.history['val_loss'], label='Validation Loss')
plt.title('LSTMATT model - Loss Function Trend')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# 4. 在测试集上评估 LSTM + Attention 模型的初步性能
lstmatt_test_loss, lstmatt_test_acc = lstmatt_model.evaluate([X_seq_test, X_static_test], y_test, verbose=2)
print(f"Initial LSTM + Attention test accuracy: {lstmatt_test_acc:.4f}")

# 5. 生成预测结果
lstmatt_y_pred = lstmatt_model.predict([X_seq_test, X_static_test], verbose=2)

# 6. 将预测结果从 one-hot 编码转换为类标签
lstmatt_y_pred_classes = lstmatt_y_pred.argmax(axis=1)
lstmatt_y_true_classes = y_test.argmax(axis=1)

# 7. 计算精确率、召回率和 F1-score
lstmatt_precision = precision_score(lstmatt_y_true_classes, lstmatt_y_pred_classes, average='weighted')
lstmatt_recall = recall_score(lstmatt_y_true_classes, lstmatt_y_pred_classes, average='weighted')
lstmatt_f1 = f1_score(lstmatt_y_true_classes, lstmatt_y_pred_classes, average='weighted')

print(f"LSTM Attention Precision: {lstmatt_precision:.4f}")
print(f"LSTM Attention Recall: {lstmatt_recall:.4f}")
print(f"LSTM Attention F1 Score: {lstmatt_f1:.4f}")

# 8. 生成并打印混淆矩阵
lstmatt_conf_matrix = confusion_matrix(lstmatt_y_true_classes, lstmatt_y_pred_classes)
print("LSTM Attention Confusion Matrix:")
print(lstmatt_conf_matrix)

# 9. 打印详细的分类报告
print("LSTM Attention Classification Report:")
print(classification_report(lstmatt_y_true_classes, lstmatt_y_pred_classes, digits=4))

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from PIL import Image

# 假设 lstmatt_y_true_classes 是 LSTM+Attention 模型的真实类标签，lstmatt_y_pred 是模型输出的预测概率
n_classes = 4

# 如果 lstmatt_y_true_classes 还没有二值化
lstmatt_y_test_binarized = label_binarize(lstmatt_y_true_classes, classes=[0, 1, 2, 3])
lstmatt_y_pred_probabilities = lstmatt_y_pred  # 假设 lstmatt_y_pred 是模型输出的概率

# 计算加权 AUC
lstmatt_weighted_auc_score = roc_auc_score(lstmatt_y_test_binarized, lstmatt_y_pred_probabilities, average='weighted', multi_class='ovr')
print(f"LSTM+Attention Model Weighted AUC: {lstmatt_weighted_auc_score:.4f}")

# 计算加权 ROC 曲线
lstmatt_fpr_weighted, lstmatt_tpr_weighted, _ = roc_curve(lstmatt_y_test_binarized.ravel(), lstmatt_y_pred_probabilities.ravel())

# 1. 单独绘制加权 ROC 曲线
plt.figure(figsize=(6, 4))
plt.plot(lstmatt_fpr_weighted, lstmatt_tpr_weighted, label=f'Weighted ROC curve (area = {lstmatt_weighted_auc_score:.4f})', color='red', linestyle='--', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', linewidth=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LSTM+Attention Model - Weighted ROC Curve')
plt.legend(loc="lower right")
plt.savefig('lstmatt_weighted_roc_curve.png')  # 保存加权 ROC 曲线图像
plt.show()

# 2. 单独绘制每个类别的 ROC 曲线
plt.figure(figsize=(6, 4))
for i in range(n_classes):
    lstmatt_fpr, lstmatt_tpr, _ = roc_curve(lstmatt_y_test_binarized[:, i], lstmatt_y_pred_probabilities[:, i])
    lstmatt_roc_auc = auc(lstmatt_fpr, lstmatt_tpr)
    plt.plot(lstmatt_fpr, lstmatt_tpr, label=f'Class {i} ROC curve (area = {lstmatt_roc_auc:.4f})')

plt.plot([0, 1], [0, 1], 'k--', linewidth=2)
plt.xlim([0, 1])
plt.ylim([0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LSTM+Attention Model - Class-wise ROC Curves')
plt.legend(loc="lower right")
plt.savefig('lstmatt_classwise_roc_curve.png')  # 保存分类 ROC 曲线图像
plt.show()

# 3. 将两张图拼接成一张左右展示的大图
# 加载图像
lstmatt_img1 = Image.open('lstmatt_weighted_roc_curve.png')
lstmatt_img2 = Image.open('lstmatt_classwise_roc_curve.png')

# 确保图像宽度相对较大，图像高度相对较小
lstmatt_img1 = lstmatt_img1.resize((int(lstmatt_img1.width * 1.5), int(lstmatt_img1.height * 0.75)))
lstmatt_img2 = lstmatt_img2.resize((int(lstmatt_img2.width * 1.5), int(lstmatt_img2.height * 0.75)))

# 创建新的图像来水平拼接
lstmatt_new_img = Image.new('RGB', (lstmatt_img1.width + lstmatt_img2.width, max(lstmatt_img1.height, lstmatt_img2.height)))
lstmatt_new_img.paste(lstmatt_img1, (0, 0))
lstmatt_new_img.paste(lstmatt_img2, (lstmatt_img1.width, 0))

# 显示拼接后的图像
lstmatt_new_img.show()

# 保存拼接后的图像
lstmatt_new_img.save('lstmatt_combined_roc_curves_side_by_side.png')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
from PIL import Image

# 假设 lstmatt_y_true_classes 是 LSTMATT 模型的真实类标签，lstmatt_y_pred 是模型的预测概率
n_classes = 4

# 如果 lstmatt_y_true_classes 还没有二值化
lstmatt_y_test_binarized = label_binarize(lstmatt_y_true_classes, classes=[0, 1, 2, 3])
lstmatt_y_pred_probabilities = lstmatt_y_pred  # 使用训练好的 LSTMATT 模型进行预测

# 1. 计算并绘制微平均和宏平均 AUC 曲线
plt.figure(figsize=(6, 4))

# 微平均 AUC
lstmatt_fpr_micro, lstmatt_tpr_micro, _ = roc_curve(lstmatt_y_test_binarized.ravel(), lstmatt_y_pred_probabilities.ravel())
lstmatt_roc_auc_micro = auc(lstmatt_fpr_micro, lstmatt_tpr_micro)
plt.plot(lstmatt_fpr_micro, lstmatt_tpr_micro, linestyle='--', color='black', lw=2, label=f'Micro-Average AUC = {lstmatt_roc_auc_micro:.4f}')

# 宏平均 AUC
all_fpr = np.unique(np.concatenate([roc_curve(lstmatt_y_test_binarized[:, i], lstmatt_y_pred_probabilities[:, i])[0] for i in range(n_classes)]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, roc_curve(lstmatt_y_test_binarized[:, i], lstmatt_y_pred_probabilities[:, i])[0],
                          roc_curve(lstmatt_y_test_binarized[:, i], lstmatt_y_pred_probabilities[:, i])[1])

mean_tpr /= n_classes
lstmatt_roc_auc_macro = auc(all_fpr, mean_tpr)
plt.plot(all_fpr, mean_tpr, linestyle='-', color='blue', lw=2, label=f'Macro-Average AUC = {lstmatt_roc_auc_macro:.4f}')

# 图形美化
plt.plot([0, 1], [0, 1], 'k--', lw=2)  # 对角线
plt.xlim([0.0, 1.0])
plt.ylim([0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LSTMATT Model - Micro & Macro Average AUC')
plt.legend(loc="lower right")
plt.savefig('lstmatt_micro_macro_auc_curve.png')  # 保存微平均和宏平均 AUC 曲线图像
plt.show()

# 2. 单独绘制每个类别的 AUC 曲线
plt.figure(figsize=(6, 4))
for i in range(n_classes):
    lstmatt_auc_score = roc_auc_score(lstmatt_y_test_binarized[:, i], lstmatt_y_pred_probabilities[:, i])
    lstmatt_fpr, lstmatt_tpr, _ = roc_curve(lstmatt_y_test_binarized[:, i], lstmatt_y_pred_probabilities[:, i])
    plt.plot(lstmatt_fpr, lstmatt_tpr, label=f'Class {i} AUC = {lstmatt_auc_score:.4f}')

# 图形美化
plt.plot([0, 1], [0, 1], 'k--', lw=2)  # 对角线
plt.xlim([0, 1])
plt.ylim([0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LSTMATT Model - classification- AUC Curves')
plt.legend(loc="lower right")
plt.savefig('lstmatt_classwise_auc_curve.png')  # 保存分类 AUC 曲线图像
plt.show()

# 3. 将两张图拼接成一张左右展示的大图
# 加载图像
lstmatt_img1 = Image.open('lstmatt_micro_macro_auc_curve.png')
lstmatt_img2 = Image.open('lstmatt_classwise_auc_curve.png')

# 调整图像大小，以便左右拼接
lstmatt_img1 = lstmatt_img1.resize((int(lstmatt_img1.width * 1), int(lstmatt_img1.height * 1)))
lstmatt_img2 = lstmatt_img2.resize((int(lstmatt_img2.width * 1), int(lstmatt_img2.height * 1)))

# 创建新的图像来水平拼接
lstmatt_new_img = Image.new('RGB', (lstmatt_img1.width + lstmatt_img2.width, max(lstmatt_img1.height, lstmatt_img2.height)))
lstmatt_new_img.paste(lstmatt_img1, (0, 0))
lstmatt_new_img.paste(lstmatt_img2, (lstmatt_img1.width, 0))

# 显示拼接后的图像
lstmatt_new_img.show()

# 保存拼接后的图像
lstmatt_new_img.save('lstmatt_combined_auc_curves_side_by_side.png')


# To make sure the colab connect successfully

In [ ]:
import time

while True:
    time.sleep(60 * 29)  # 每 29 分钟执行一次
    print("Running to keep the session alive...")

# Hyperparameter tuning, 下面的还未修改

In [ ]:
def build_lstm_model_ajust(units1=64, units2=32, optimizer='adam'):
    # 序列输入
    sequence_input = Input(shape=(X_seq_train_resampled.shape[1], X_seq_train_resampled.shape[2]), name='sequence_input')
    # 静态输入
    static_input = Input(shape=(X_static_train.shape[1],), name='static_input')

    # # Attention 层
    # attention = Dense(1, activation='tanh')(sequence_input)
    # attention = Flatten()(attention)
    # attention = Activation('softmax')(attention)
    # attention = RepeatVector(X_seq_train_resampled.shape[2])(attention)
    # attention = Permute([2, 1])(attention)

    # LSTM 层
    lstm_input = Multiply()([sequence_input, attention])
    x = LSTM(units1, return_sequences=True)(lstm_input)
    x = Dropout(0.2)(x)
    x = LSTM(units2)(x)
    x = Dropout(0.2)(x)

    # 将静态特征与 LSTM 输出连接
    x = Concatenate()([x, static_input])

    # 全连接层
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(4, activation='softmax')(x)

    # 构建模型
    model = Model(inputs=[sequence_input, static_input], outputs=output)

    # 编译模型
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# 使用 KerasRegressor 包装模型，并将需要调优的参数作为参数传递
model = KerasRegressor(model=build_lstm_model, verbose=1)

# set param_grid，and confirm units2 is a half of units1
# param_grid = {
#     'batch_size': [64, 128],
#     'epochs': [100, 150],
#     'model__units1': [32, 64, 128],  # 注意：使用 `model__` 前缀来传递给模型构建函数
#     'model__units2': [16, 32, 64],   # 同样，第二层LSTM的神经元数量使用 `model__`
#     'model__optimizer': ['adam', 'rmsprop']  # 优化器
# }
# 自定义参数网格：确保 units2 始终为 units1 的一半
# 设置参数网格
param_grid = []

for units1 in [64, 128]:
    param_grid.append({
        'batch_size': [64, 128],
        'epochs': [100, 150],
        'model__units1': [units1],
        'model__units2': [units1 // 2],  # 确保 units2 是 units1 的一半
        'model__optimizer': ['adam', 'rmsprop']
    })

# 设置交叉验证策略

In [ ]:
# # fit model
# history = model.fit([X_seq_train_resampled, X_static_train_resampled], y_train_resampled,
#                     batch_size=64, epochs=10,
#                     # validation_split=0.2,
#                     callbacks=[early_stopping],
#                     # class_weight=class_weights_dict,
#                     verbose=2)

In [ ]:
# fit model
history = model.fit(
      [X_seq_train, X_static_train], y_train,
      epochs=100,
      batch_size=64,
      validation_data=([X_seq_test, X_static_test], y_test),
      callbacks=[early_stopping],  # 添加 EarlyStopping 回调
      verbose=2
)

# 评估模型
loss, accuracy = model.evaluate([X_seq_test, X_static_test], y_test)
print(f"测试集准确率: {accuracy:.4f}")

# 预测并输出性能指标
y_pred = model.predict([X_seq_test, X_static_test])
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print("\n混淆矩阵:")
print(confusion_matrix(y_true_classes, y_pred_classes))

print("\n分类报告:")
print(classification_report(y_true_classes, y_pred_classes))

# **下面的是参考**

In [ ]:
# ### 保留，这个是正确的

# # 选择需要标准化的特征
# features_to_scale = ['Bicarbonate', 'Creatinine', 'Glucose', 'Oxygen Saturation', 'Platelet Count', 'Potassium']

# # 标准化数据
# scaler = StandardScaler()
# df_imputed[features_to_scale] = scaler.fit_transform(df_imputed[features_to_scale])

# # 构建 LSTM 时间序列数据
# def create_sequences(data, time_steps=22):
#     X, X_static, y = [], [], []
#     for _, group in data.groupby('subject_id'):
#         feature_data = group[features_to_scale].values
#         static_data = group[['gender', 'aki_age', 'bmi']].values
#         target_data = group['survival_category'].values

#         if len(feature_data) >= time_steps:
#             for i in range(len(feature_data) - time_steps + 1):
#                 X.append(feature_data[i:i+time_steps])
#                 X_static.append(static_data[i+time_steps-1])  # 取最后一个时间步的静态特征
#                 y.append(target_data[i+time_steps-1])
#     return np.array(X), np.array(X_static), np.array(y)

# # 生成数据
# time_steps = 22
# X_sequence, X_static, y = create_sequences(df_imputed, time_steps)

# # 转换目标标记为 one-hot 编码
# y_categorical = to_categorical(y, num_classes=4)

# # 分割数据集
# X_seq_train, X_seq_test, X_static_train, X_static_test, y_train, y_test = train_test_split(
#     X_sequence, X_static, y_categorical, test_size=0.2, random_state=42)

# # 确保 X_train 和 y_train 是浮点型
# X_seq_train = np.array(X_seq_train, dtype=np.float32)
# X_seq_test = np.array(X_seq_test, dtype=np.float32)
# X_static_train = np.array(X_static_train, dtype=np.float32)
# X_static_test = np.array(X_static_test, dtype=np.float32)
# y_train = np.array(y_train, dtype=np.float32)
# y_test = np.array(y_test, dtype=np.float32)

# # 构建 LSTM 模型
# def build_lstm_model():
#     sequence_input = Input(shape=(X_seq_train.shape[1], X_seq_train.shape[2]), name='sequence_input')
#     static_input = Input(shape=(X_static_train.shape[1],), name='static_input')

#     # LSTM 层
#     x = LSTM(64, return_sequences=True)(sequence_input)
#     x = Dropout(0.2)(x)
#     x = LSTM(32)(x)
#     x = Dropout(0.2)(x)

#     # 将静态特征与 LSTM 输出连接
#     x = Concatenate()([x, static_input])

#     # 全连接层
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     output = Dense(4, activation='softmax')(x)

#     # 构建模型
#     model = Model(inputs=[sequence_input, static_input], outputs=output)

#     return model

# # 实例化和编译模型
# model = build_lstm_model()
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # 创建 EarlyStopping 回调实例
# early_stopping = EarlyStopping(
#     monitor='val_loss',  # 监控验证损失
#     patience=5,          # 如果验证损失在 5 个 epoch 内没有改善，则停止训练
#     verbose=1,           # 显示日志信息
#     mode='min',          # 验证损失要最小化
#     restore_best_weights=True  # 恢复模型到验证集表现最好的权重
# )

# # 训练模型
# model.fit(
#     [X_seq_train, X_static_train], y_train,
#     epochs=50,
#     batch_size=32,
#     validation_data=([X_seq_test, X_static_test], y_test),
#     callbacks=[early_stopping],  # 添加 EarlyStopping 回调
#     verbose=1
# )

# # 评估模型
# loss, accuracy = model.evaluate([X_seq_test, X_static_test], y_test)
# print(f"测试集准确率: {accuracy:.4f}")

# # 预测并输出性能指标
# y_pred = model.predict([X_seq_test, X_static_test])
# y_pred_classes = np.argmax(y_pred, axis=1)
# y_true_classes = np.argmax(y_test, axis=1)

# print("\n混淆矩阵:")
# print(confusion_matrix(y_true_classes, y_pred_classes))

# print("\n分类报告:")
# print(classification_report(y_true_classes, y_pred_classes))

# 引入类别不平衡的以后

In [ ]:
# 构建 LSTM 时间序列数据
def create_sequences(data, time_steps=22):
    X, X_static, y = [], [], []
    for _, group in data.groupby('subject_id'):
        feature_data = group[features_to_scale].values
        static_data = group[['gender', 'aki_age', 'bmi']].values
        target_data = group['survival_category'].values

        if len(feature_data) >= time_steps:
            for i in range(len(feature_data) - time_steps + 1):
                X.append(feature_data[i:i+time_steps])
                X_static.append(static_data[i+time_steps-1])  # 取最后一个时间步的静态特征
                y.append(target_data[i+time_steps-1])
    return np.array(X), np.array(X_static), np.array(y)

# 生成数据
time_steps = 22
X_sequence, X_static, y = create_sequences(df_imputed, time_steps)

# 转换目标标记为 one-hot 编码
y_categorical = to_categorical(y, num_classes=4)

# 分割数据集
X_seq_train, X_seq_test, X_static_train, X_static_test, y_train, y_test = train_test_split(
    X_sequence, X_static, y_categorical, test_size=0.2, random_state=42)

# 确保 X_train 和 y_train 是浮点型
X_seq_train = np.array(X_seq_train, dtype=np.float32)
X_seq_test = np.array(X_seq_test, dtype=np.float32)
X_static_train = np.array(X_static_train, dtype=np.float32)
X_static_test = np.array(X_static_test, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

In [ ]:
# Reshape for SMOTE
X_seq_train_reshaped = X_seq_train.reshape(X_seq_train.shape[0], -1)

# 修改部分：将静态特征 X_static_train 同步进行 SMOTE 处理
# 先将静态特征扩展维度，使得可以与序列特征进行拼接
X_static_train_reshaped = X_static_train.reshape(X_static_train.shape[0], -1)

# Combine sequence and static features
combined_X_train = np.hstack([X_seq_train_reshaped, X_static_train_reshaped])

# Apply SMOTE
smote = SMOTE(random_state=42)
combined_X_train_resampled, y_train_resampled = smote.fit_resample(combined_X_train, np.argmax(y_train, axis=1))

# Split the resampled data back into sequence and static features
X_seq_train_resampled = combined_X_train_resampled[:, :X_seq_train_reshaped.shape[1]]
X_static_train_resampled = combined_X_train_resampled[:, X_seq_train_reshaped.shape[1]:]

# Reshape sequence data back to its original shape
X_seq_train_resampled = X_seq_train_resampled.reshape(-1, time_steps, len(features_to_scale))
y_train_resampled = to_categorical(y_train_resampled, num_classes=4)

# 检查序列特征和静态特征的形状
print(X_seq_train_resampled.shape)
print(X_static_train_resampled.shape)

# # 构建 LSTM 模型
# def build_lstm_model():
#     sequence_input = Input(shape=(X_seq_train_resampled.shape[1], X_seq_train_resampled.shape[2]), name='sequence_input')
#     static_input = Input(shape=(X_static_train_resampled.shape[1],), name='static_input')

#     # Attention 层
#     attention = Dense(1, activation='tanh')(sequence_input)
#     attention = Flatten()(attention)
#     attention = Activation('softmax')(attention)
#     attention = RepeatVector(X_seq_train_resampled.shape[2])(attention)
#     attention = Permute([2, 1])(attention)

#     # LSTM 层
#     lstm_input = Multiply()([sequence_input, attention])
#     x = LSTM(64, return_sequences=True)(lstm_input)
#     x = Dropout(0.2)(x)
#     x = LSTM(32)(x)
#     x = Dropout(0.2)(x)

#     # 将静态特征与 LSTM 输出连接
#     x = Concatenate()([x, static_input])

#     # 全连接层
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     output = Dense(4, activation='softmax')(x)

#     # 构建模型
#     model = Model(inputs=[sequence_input, static_input], outputs=output)

#     return model
#简单的模型
def build_lstm_model():
    sequence_input = Input(shape=(X_seq_train_resampled.shape[1], X_seq_train_resampled.shape[2]), name='sequence_input')
    static_input = Input(shape=(X_static_train_resampled.shape[1],), name='static_input')

    # LSTM 层
    x = LSTM(32, return_sequences=True)(sequence_input)
    x = Dropout(0.2)(x)
    x = LSTM(16)(x)
    x = Dropout(0.2)(x)

    # 将静态特征与 LSTM 输出连接
    x = Concatenate()([x, static_input])

    # 全连接层
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(4, activation='softmax')(x)

    model = Model(inputs=[sequence_input, static_input], outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
# 重新编译模型，使用较低的学习率
model = build_lstm_model()

# 尝试训练模型
history = model.fit([X_seq_train_resampled, X_static_train_resampled], y_train_resampled,
                    batch_size=64, epochs=10,
                    # validation_split=0.2,
                    callbacks=[early_stopping],
                    class_weight=class_weights_dict, verbose=2)

In [ ]:
import numpy as np
print("Training labels distribution:", np.sum(y_train_resampled, axis=0))
# print("Validation labels distribution:", np.sum(y_val_resampled, axis=0))


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
# from scikit.learn import KerasClassifier
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import parallel_backend


In [ ]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print(f"{mean} (+/-{std}) with: {param}")


In [ ]:
# from scikeras.wrappers import KerasRegressor
# from sklearn.model_selection import GridSearchCV, KFold
# from keras.models import Sequential
# from keras.layers import LSTM, Dense
# import numpy as np

# # 定义模型的构建函数，使用**kwargs接收参数
# # def build_lstm_model(units1=64, units2=32, optimizer='adam'):
# #     model = Sequential()
# #     # 使用 Input 层来定义输入形状
# #     model.add(Input(shape=(X_seq_train_resampled.shape[1], X_seq_train_resampled.shape[2])))
# #     # 添加 LSTM 层
# #     model.add(LSTM(units=units1, return_sequences=True))
# #     model.add(LSTM(units=units2))
# #     # 添加输出层
# #     model.add(Dense(1))  # 回归问题通常输出一个值，因此输出层为 Dense(1)
# #     # 编译模型
# #     model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_squared_error'])
# #     return model


# def build_lstm_model(units1=64, units2=32, optimizer='adam'):
#     # 序列输入
#     sequence_input = Input(shape=(X_seq_train_resampled.shape[1], X_seq_train_resampled.shape[2]), name='sequence_input')
#     # 静态输入
#     static_input = Input(shape=(X_static_train.shape[1],), name='static_input')

#     # Attention 层
#     attention = Dense(1, activation='tanh')(sequence_input)
#     attention = Flatten()(attention)
#     attention = Activation('softmax')(attention)
#     attention = RepeatVector(X_seq_train_resampled.shape[2])(attention)
#     attention = Permute([2, 1])(attention)

#     # LSTM 层
#     lstm_input = Multiply()([sequence_input, attention])
#     x = LSTM(units1, return_sequences=True)(lstm_input)
#     x = Dropout(0.2)(x)
#     x = LSTM(units2)(x)
#     x = Dropout(0.2)(x)

#     # 将静态特征与 LSTM 输出连接
#     x = Concatenate()([x, static_input])

#     # 全连接层
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     output = Dense(4, activation='softmax')(x)

#     # 构建模型
#     model = Model(inputs=[sequence_input, static_input], outputs=output)

#     # 编译模型
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

#     return model

# # 使用 KerasRegressor 包装模型，并将需要调优的参数作为参数传递
# model = KerasRegressor(model=build_lstm_model, verbose=0)

# # 设置 param_grid，其中参数名称必须与 build_lstm_model 函数的参数名匹配
# # 设置 param_grid，其中参数名称必须与 build_lstm_model 函数的参数名匹配
# # param_grid = {
# #     'batch_size': [64, 128],
# #     'epochs': [100, 150],
# #     'model__units1': [32, 64, 128],  # 注意：使用 `model__` 前缀来传递给模型构建函数
# #     'model__units2': [16, 32, 64],   # 同样，第二层LSTM的神经元数量使用 `model__`
# #     'model__optimizer': ['adam', 'rmsprop']  # 优化器
# # }
# # 自定义参数网格：确保 units2 始终为 units1 的一半
# # 设置参数网格
# param_grid = []

# for units1 in [64, 128]:
#     param_grid.append({
#         'batch_size': [64, 128],
#         'epochs': [100, 150],
#         'model__units1': [units1],
#         'model__units2': [units1 // 2],  # 确保 units2 是 units1 的一半
#         'model__optimizer': ['adam', 'rmsprop']
#     })

# # 设置交叉验证策略
# kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# # 添加早停回调
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# # 创建 GridSearchCV 对象，
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=kfold, verbose=2, refit=True)

# # 使用正确的标签数据进行拟合
# # 使用正确的标签数据进行拟合，并传递两个输入
# inputs = [X_seq_train_resampled, X_static_train_resampled]


# grid_result = grid.fit(inputs, y_train_resampled,# class_weight=class_weights_dict,  # 应用类别权重
#                        validation_split=0.2, callbacks=[early_stopping])

# # 输出结果
# print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")


In [ ]:
import itertools
import numpy as np
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

# 定义参数网格
param_grid = {
    'units1': [64, 128],
    'batch_size': [64, 128],
    'epochs': [100, 150],
    'optimizer': ['adam', 'rmsprop']
}

# 准备存储最优模型及其性能
best_score = -1
best_params = None
best_model = None

# 手动遍历所有参数组合
param_combinations = list(itertools.product(param_grid['units1'],
                                            param_grid['batch_size'],
                                            param_grid['epochs'],
                                            param_grid['optimizer']))

for combination in param_combinations:
    units1, batch_size, epochs, optimizer = combination
    units2 = units1 // 2  # 确保 units2 是 units1 的一半

    # 构建模型
    model = build_lstm_model()  # 使用之前定义的 build_lstm_model 函数

    # 编译模型
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # 创建早停回调
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # 训练模型
    history = model.fit([X_seq_train_resampled, X_static_train_resampled], y_train_resampled,
                        batch_size=batch_size, epochs=epochs,
                        validation_split=0.2, callbacks=[early_stopping],
                        class_weight=class_weights_dict, verbose=2)

    # 在验证集上评估模型
    val_loss, val_acc = model.evaluate([X_seq_val_resampled, X_static_val_resampled], y_val_resampled, verbose=2)

    # 如果当前参数组合表现更好，则更新最佳参数和模型
    if val_acc > best_score:
        best_score = val_acc
        best_params = {
            'units1': units1,
            'units2': units2,
            'batch_size': batch_size,
            'epochs': epochs,
            'optimizer': optimizer
        }
        best_model = model

# 输出最佳参数组合和对应的性能
print(f"Best score: {best_score} with parameters: {best_params}")


In [ ]:
# 实例化和编译模型
model = build_lstm_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 创建 EarlyStopping 回调实例
early_stopping = EarlyStopping(
    monitor='val_loss',  # 监控验证损失
    patience=5,          # 如果验证损失在 5 个 epoch 内没有改善，则停止训练
    verbose=1,           # 显示日志信息
    mode='min',          # 验证损失要最小化
    restore_best_weights=True  # 恢复模型到验证集表现最好的权重
)

# 训练模型
model.fit(
    [X_seq_train_resampled, X_static_train[:X_seq_train_resampled.shape[0]]], y_train_resampled,
    epochs=80,
    batch_size=32,
    validation_data=([X_seq_test, X_static_test[:X_seq_test.shape[0]]], y_test),
    class_weight=class_weights_dict,  # 应用类别权重
    callbacks=[early_stopping],  # 添加 EarlyStopping 回调
    verbose=1
)

# 评估模型
loss, accuracy = model.evaluate([X_seq_test, X_static_test], y_test)
print(f"测试集准确率: {accuracy:.4f}")

# 预测并输出性能指标
y_pred = model.predict([X_seq_test, X_static_test])
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print("\n混淆矩阵:")
print(confusion_matrix(y_true_classes, y_pred_classes))

print("\n分类报告:")
print(classification_report(y_true_classes, y_pred_classes))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate, Attention
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

# 假设df_imputed是已经填补好缺失值的数据集

# 选择需要标准化的特征
features_to_scale = ['Bicarbonate', 'Creatinine', 'Glucose', 'Oxygen Saturation', 'Platelet Count', 'Potassium']

# 标准化数据
scaler = StandardScaler()
df_imputed[features_to_scale] = scaler.fit_transform(df_imputed[features_to_scale])

# 构建 LSTM 时间序列数据
def create_sequences(data, time_steps=22):
    X, X_static, y = [], [], []
    for _, group in data.groupby('subject_id'):
        feature_data = group[features_to_scale].values
        static_data = group[['gender', 'aki_age', 'bmi']].values
        target_data = group['survival_category'].values

        if len(feature_data) >= time_steps:
            for i in range(len(feature_data) - time_steps + 1):
                X.append(feature_data[i:i+time_steps])
                X_static.append(static_data[i+time_steps-1])  # 取最后一个时间步的静态特征
                y.append(target_data[i+time_steps-1])
    return np.array(X), np.array(X_static), np.array(y)

# 生成数据
time_steps = 22
X_sequence, X_static, y = create_sequences(df_imputed, time_steps)

# 转换目标标记为 one-hot 编码
y_categorical = to_categorical(y, num_classes=4)

# 分割数据集
X_seq_train, X_seq_test, X_static_train, X_static_test, y_train, y_test = train_test_split(
    X_sequence, X_static, y_categorical, test_size=0.2, random_state=42)

# 确保 X_train 和 y_train 是浮点型
X_seq_train = np.array(X_seq_train, dtype=np.float32)
X_seq_test = np.array(X_seq_test, dtype=np.float32)
X_static_train = np.array(X_static_train, dtype=np.float32)
X_static_test = np.array(X_static_test, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

In [ ]:
from imblearn.over_sampling import SMOTE
import numpy as np

# Assuming you have already defined X_seq_train and y_train
n_samples, time_steps, n_features = X_seq_train.shape

# Flatten the sequence data
X_seq_train_flat = X_seq_train.reshape(n_samples, -1)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_seq_train_resampled_flat, y_train_resampled = smote.fit_resample(X_seq_train_flat, y_train)

# Reshape back to the original sequence format
X_seq_train_resampled = X_seq_train_resampled_flat.reshape(-1, time_steps, n_features)

# Adjust X_static_train to match the resampled sequence data
num_resampled_samples = len(X_seq_train_resampled)
num_static_samples = len(X_static_train)

# Calculate the required number of repetitions
num_repeats = num_resampled_samples // num_static_samples
remainder = num_resampled_samples % num_static_samples

# Repeat and concatenate the static data to match the sequence data samples
X_static_train_resampled = np.vstack((np.tile(X_static_train, (num_repeats, 1)),
                                      X_static_train[:remainder]))

print(X_static_train_resampled.shape)
print(X_seq_train_resampled.shape)

In [ ]:
print(X_static_train_resampled.shape)
print(X_seq_train_resampled.shape)

In [ ]:
# 构建 LSTM 模型
def build_lstm_model():
    sequence_input = Input(shape=(X_seq_train_resampled.shape[1], X_seq_train_resampled.shape[2]), name='sequence_input')
    static_input = Input(shape=(X_static_train_resampled.shape[1],), name='static_input')

    # LSTM 层
    x = LSTM(64, return_sequences=True)(sequence_input)
    x = Dropout(0.2)(x)

    # 自注意力层
    attn_layer = Attention()([x, x])

    x = LSTM(32)(attn_layer)
    x = Dropout(0.2)(x)

    # 将静态特征与 LSTM 输出连接
    x = Concatenate()([x, static_input])

    # 全连接层
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(4, activation='softmax')(x)

    # 构建模型
    model = Model(inputs=[sequence_input, static_input], outputs=output)

    return model

# 实例化和编译模型
model = build_lstm_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 创建 EarlyStopping 回调实例
early_stopping = EarlyStopping(
    monitor='val_loss',  # 监控验证损失
    patience=5,          # 如果验证损失在 5 个 epoch 内没有改善，则停止训练
    verbose=1,           # 显示日志信息
    mode='min',          # 验证损失要最小化
    restore_best_weights=True  # 恢复模型到验证集表现最好的权重
)

# 训练模型
model_details=model.fit(
    [X_seq_train_resampled, X_static_train_resampled], y_train_resampled,
    epochs=80,
    batch_size=64,
    validation_data=([X_seq_test, X_static_test], y_test),
    callbacks=[early_stopping],  # 添加 EarlyStopping 回调
    verbose=1
)

In [ ]:
# 评估模型
loss, accuracy = model.evaluate([X_seq_test, X_static_test], y_test)
print(f"测试集准确率: {accuracy:.4f}")

# 预测并输出性能指标
y_pred = model.predict([X_seq_test, X_static_test])
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print("\n混淆矩阵:")
print(confusion_matrix(y_true_classes, y_pred_classes))

print("\n分类报告:")
print(classification_report(y_true_classes, y_pred_classes))

In [ ]:
import matplotlib.pyplot as plt

# 绘制训练损失和验证损失的图表
plt.figure(figsize=(12, 6))
plt.plot(model_details.history['loss'], label='Training Loss')
plt.plot(model_details.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
print(X_seq_train.shape)
print(X_static_train.shape)

# 下面的是之前的

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import classification_report, confusion_matrix
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.utils import to_categorical

In [ ]:
# # 选择需要标准化的特征
# features_to_scale = ['Bicarbonate', 'Creatinine', 'Glucose', 'Oxygen Saturation', 'Platelet Count', 'Potassium']

# # 标准化数据
# scaler = StandardScaler()
# df_all[features_to_scale] = scaler.fit_transform(df_all[features_to_scale])

In [ ]:
# # 构建 LSTM 时间序列数据
# def create_sequences(data, time_steps=22):
#     X, y = [], []
#     for _, group in data.groupby('subject_id'):
#         feature_data = group[features_to_scale + ['gender', 'aki_age', 'bmi']].values
#         target_data = group['survival_category'].values
#         if len(feature_data) >= time_steps:
#             for i in range(len(feature_data) - time_steps + 1):
#                 X.append(feature_data[i:i+time_steps])
#                 y.append(target_data[i+time_steps-1])
#     return np.array(X), np.array(y)

# # 生成数据
# time_steps = 22
# X, y = create_sequences(df_all, time_steps)

# # 转换目标变量为 one-hot 编码
# y_categorical = to_categorical(y, num_classes=4)

# # 分割数据集
# X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)
# 构建 LSTM 时间序列数据
def create_sequences(data, time_steps=22):
    X, X_static, y = [], [], []
    for _, group in data.groupby('subject_id'):
        feature_data = group[features_to_scale].values
        static_data = group[['gender', 'aki_age', 'bmi']].values
        target_data = group['survival_category'].values

        if len(feature_data) >= time_steps:
            for i in range(len(feature_data) - time_steps + 1):
                X.append(feature_data[i:i+time_steps])
                X_static.append(static_data[i+time_steps-1])  # 取最后一个时间步的静态特征
                y.append(target_data[i+time_steps-1])
    return np.array(X), np.array(X_static), np.array(y)

# 生成数据
time_steps = 22
X_sequence, X_static, y = create_sequences(df_all, time_steps)

# 转换目标标记为 one-hot 编码
y_categorical = to_categorical(y, num_classes=4)

# 分割数据集
X_seq_train, X_seq_test, X_static_train, X_static_test, y_train, y_test = train_test_split(
    X_sequence, X_static, y_categorical, test_size=0.2, random_state=42)


In [ ]:
# 确保 X_train 和 y_train 是浮点型
X_seq_train = np.array(X_seq_train, dtype=np.float32)
X_seq_test = np.array(X_seq_test, dtype=np.float32)
X_static_train = np.array(X_static_train, dtype=np.float32)
X_static_test = np.array(X_static_test, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

#import numpy as np
#X_train 和 y_train 是浮点型
# X_train = np.array(X_train, dtype=np.float32)
# X_test = np.array(X_test, dtype=np.float32)

# # y_train 和 y_test 已经是 one-hot 编码，确保为浮点型
# y_train = np.array(y_train, dtype=np.float32)
# y_test = np.array(y_test, dtype=np.float32)

In [ ]:
# # 构建 LSTM 模型
# model = Sequential([
#     LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
#     Dropout(0.2),
#     LSTM(32),
#     Dropout(0.2),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(4, activation='softmax')  # 输出4个类别的概率
# ])

# 构建 LSTM 模型
def build_lstm_model():
    sequence_input = Input(shape=(X_seq_train.shape[1], X_seq_train.shape[2]), name='sequence_input')
    static_input = Input(shape=(X_static_train.shape[1],), name='static_input')

    # LSTM 层
    x = LSTM(64, return_sequences=True)(sequence_input)
    x = Dropout(0.2)(x)
    x = LSTM(32)(x)
    x = Dropout(0.2)(x)

    # 将静态特征与 LSTM 输出连接
    x = Concatenate()([x, static_input])

    # 全连接层
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(4, activation='softmax')(x)

    # 构建模型
    model = Model(inputs=[sequence_input, static_input], outputs=output)

    return model


# 实例化和编译模型
model = build_lstm_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 创建 EarlyStopping 回调实例
early_stopping = EarlyStopping(
    monitor='val_loss',  # 监控验证损失
    patience=5,          # 如果验证损失在 5 个 epoch 内没有改善，则停止训练
    verbose=1,           # 显示日志信息
    mode='min',          # 验证损失要最小化
    restore_best_weights=True  # 恢复模型到验证集表现最好的权重
)

# 训练模型
model.fit(
    [X_seq_train, X_static_train], y_train,
    epochs=50,
    batch_size=32,
    validation_data=([X_seq_test, X_static_test], y_test),
    callbacks=[early_stopping],  # 添加 EarlyStopping 回调
    verbose=1
)


# model = build_lstm_model()

# # 编译模型
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # 创建 EarlyStopping 回调实例
# early_stopping = EarlyStopping(
#     monitor='val_loss',  # 监控验证损失
#     patience=5,          # 如果验证损失在 5 个 epoch 内没有改善，则停止训练
#     verbose=1,           # 显示日志信息
#     mode='min',          # 验证损失要最小化
#     restore_best_weights=True  # 恢复模型到验证集表现最好的权重
# )

# # 训练模型
# model.fit(X_train, y_train,
#           epochs=50,
#           batch_size=32,
#           validation_data=(X_test, y_test),
#           callbacks=[early_stopping],  # 添加 EarlyStopping 回调
#           verbose=1)

# # 评估模型
# loss, accuracy = model.evaluate(X_test, y_test)
# print(f"测试集准确率: {accuracy:.4f}")

# # 预测并输出性能指标
# y_pred = model.predict(X_test)
# y_pred_classes = np.argmax(y_pred, axis=1)
# y_true_classes = np.argmax(y_test, axis=1)

# print("\n混淆矩阵:")
# print(confusion_matrix(y_true_classes, y_pred_classes))

# print("\n分类报告:")
# print(classification_report(y_true_classes, y_pred_classes))


In [ ]:
# 评估模型
loss, accuracy = model.evaluate([X_seq_test, X_static_test], y_test)
print(f"测试集准确率: {accuracy:.4f}")

# 预测并输出性能指标
y_pred = model.predict([X_seq_test, X_static_test])
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print("\n混淆矩阵:")
print(confusion_matrix(y_true_classes, y_pred_classes))

print("\n分类报告:")
print(classification_report(y_true_classes, y_pred_classes))

# 后面是第二种缺失值处理的方法，区别在于先删除50%以上缺失数据的

In [ ]:
df_all1 = df_pivot.copy()

df_all1

In [ ]:
# Set the threshold for missing values

threshold = 0.5  # Set the threshold for missing values

df_all1 = df_all1.dropna(thresh=int(threshold * df_all1.shape[1]))

df_all1
